In [1]:
import os

In [2]:
%pwd

'd:\\Projects\\Text_Summarizer\\research'

In [3]:
os.chdir('../')

In [5]:
os.listdir()

['.git',
 '.github',
 '.gitignore',
 'app.py',
 'artifacts',
 'config',
 'Dockerfile',
 'LICENSE',
 'logs',
 'main.py',
 'params.yaml',
 'README.md',
 'requirements.txt',
 'research',
 'setup.py',
 'src',
 'template.py',
 'Text_Summarization.ipynb',
 'venv']

In [6]:
%pwd

'd:\\Projects\\Text_Summarizer'

In [54]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: str
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int



In [55]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories
from src.textSummarizer.entity import DataTransformationConfig

In [56]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):
            
            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
          config = self.config.model_trainer
          params = self.params.TrainingArguments

          create_directories([config.root_dir])

          model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_ckpt = config.model_ckpt,
            num_train_epochs = params.num_train_epochs,
            warmup_steps = params.warmup_steps,
            per_device_train_batch_size = params.per_device_train_batch_size,
            weight_decay = params.weight_decay,
            logging_steps = params.logging_steps,
            evaluation_strategy = params.evaluation_strategy,
            eval_steps = params.eval_steps,
            save_steps = params.save_steps,
            gradient_accumulation_steps = params.gradient_accumulation_steps
        )

          return model_trainer_config

In [57]:
#components

from transformers import TrainingArguments, Trainer
from transformers import DataCollatorForSeq2Seq
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset, load_from_disk
import torch



In [61]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        dataset_samsum_pt = load_from_disk(self.config.data_path)

        trainer_args = TrainingArguments(
            outpur_dir = self.config.root_dir,
            num_train_epochs= 1,
            warmup_steps= 500,
            per_device_train_batch_size= 1,
            per_device_eval_batch_size= 1,
            weight_decay= 0.01,
            logging_steps= 10,
            evaluation_strategy='steps',
            eval_steps= 500,
            save_steps= 1e6,
            gradient_accumulation_steps= 16
        )

        trainer = Trainer(model = model_pegasus, args = trainer_args,
                          tokenizer = tokenizer,
                          data_collator = seq2seq_data_collator,
                          train_dataset = dataset_samsum_pt["test"],
                          eval_dataset = dataset_samsum_pt["validation"])
        
        trainer.train()

        model_pegasus.save_pretrained(os.path.join(self.config.root_dir, "pegasus-samsum-model"))
        tokenizer.save_pretrained(os.path.join(self.config.root_dir, "tokenizer"))


In [62]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()

except Exception as e:
    raise e

[2025-03-30 21:14:37,759:  INFO: yaml file loaded successfully: config\config.yaml]
[2025-03-30 21:14:37,766:  INFO: yaml file loaded successfully: params.yaml]
[2025-03-30 21:14:37,768:  INFO: Directory created at artifacts]
[2025-03-30 21:14:37,770:  INFO: Directory created at artifacts/model_trainer]


d:\Projects\Text_Summarizer\venv\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ASUS\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/peg

FileNotFoundError: Directory artifacts/data_transformation/samsum_dataset not found